<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_multiple_stock_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
from finrl.trade.backtest import backtest_stats, get_baseline, backtest_plot

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [6]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [7]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [8]:
print(config.SRI_KEHATI_TICKER)

['AALI.JK', 'ADHI.JK', 'ASII.JK', 'BBCA.JK', 'BBNI.JK', 'BBRI.JK', 'BBTN.JK', 'BMRI.JK', 'BSDE.JK', 'INDF.JK', 'JPFA.JK', 'JSMR.JK', 'KLBF.JK', 'PGAS.JK', 'PJAA.JK', 'PPRO.JK', 'SIDO.JK', 'SMGR.JK', 'TINS.JK', 'TLKM.JK', 'UNTR.JK', 'UNVR.JK', 'WIKA.JK', 'WSKT.JK', 'WTON.JK']


In [9]:
df = YahooDownloader(start_date = config.START_DATE,
                     end_date = '2021-01-19',
                     ticker_list = config.SRI_KEHATI_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2000-09-05,455.351990,455.351990,441.553986,266.565033,3712344.0,UNTR.JK,1
1,2000-09-06,441.553986,464.552002,436.954010,269.284821,6793100.0,UNTR.JK,2
2,2000-09-07,450.752991,459.951996,446.153015,269.284821,5011393.0,UNTR.JK,3
3,2000-09-08,450.752991,455.351990,446.153015,266.565033,2277955.0,UNTR.JK,4
4,2000-09-11,441.553986,441.553986,427.755005,255.684875,978363.0,UNTR.JK,0


In [10]:
df.shape

(92694, 8)

# Part 4: Preprocess Data for Traditional Markets
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [10]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [51]:
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2000-09-05,455.352478,455.352478,441.553925,266.785400,3712343.0,UNTR.JK,1,0.000000,271.996328,264.296702,100.000000,66.666667,100.000000,266.785400,266.785400,0.0
1,2000-09-06,441.553925,464.551514,436.954376,269.507629,6793100.0,UNTR.JK,2,0.061076,271.996328,264.296702,100.000000,66.666667,100.000000,268.146515,268.146515,0.0
2,2000-09-07,450.752960,459.951996,446.153442,269.507629,5011392.0,UNTR.JK,3,0.077823,271.743579,265.456860,100.000000,85.853710,100.000000,268.600220,268.600220,0.0
3,2000-09-08,450.752960,455.352478,446.153442,266.785400,2277955.0,UNTR.JK,4,-0.014753,271.289874,265.003156,48.305572,-24.786612,100.000000,268.146515,268.146515,0.0
4,2000-09-11,441.553925,441.553925,427.755341,255.896164,978362.0,UNTR.JK,0,-0.498054,276.986591,254.406298,15.388141,-166.666667,41.910966,265.696445,265.696445,0.0


# Load Crypto Data

We load crypto data from [Kaggle](https://www.kaggle.com/jessevent/all-crypto-currencies). Hopefully we can pre-process just the same as traditional market data. The only info we should need are high,low,close,volume.

In [115]:
cr_df = pd.read_csv('~/crypto-markets_top30.csv')
cr_df.head()


,tic,date,open,high,low,close,volume
0,BTC,2013-04-28,135.30,135.98,132.10,134.21,0.0
1,BTC,2013-04-29,134.44,147.49,134.00,144.54,0.0
2,BTC,2013-04-30,144.00,146.93,134.05,139.00,0.0
3,BTC,2013-05-01,139.00,139.89,107.72,116.99,0.0
4,BTC,2013-05-02,116.38,125.60,92.28,105.21,0.0


In [116]:
cr_fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

cr_processed = fe.preprocess_data(cr_df)


Successfully added technical indicators
Successfully added turbulence index


In [117]:
cr_processed.head()

,tic,date,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,BTC,2013-04-28,135.30,135.98,132.10,134.21,0.0,0.000000,153.983826,124.766174,100.000000,66.666667,100.0,134.210,134.210,0.0
1,LTC,2013-04-28,4.30,4.40,4.18,4.35,0.0,0.000000,153.983826,124.766174,100.000000,66.666667,100.0,4.350,4.350,0.0
2,BTC,2013-04-29,134.44,147.49,134.00,144.54,0.0,0.231763,153.983826,124.766174,100.000000,66.666667,100.0,139.375,139.375,0.0
3,LTC,2013-04-29,4.37,4.57,4.23,4.38,0.0,0.000673,4.407426,4.322574,100.000000,66.666667,100.0,4.365,4.365,0.0
4,BTC,2013-04-30,144.00,146.93,134.05,139.00,0.0,0.123517,149.589072,128.910928,64.317152,28.095583,100.0,139.250,139.250,0.0
